# Classification and Regression

In [1]:
import math
import time
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow_probability import edward2 as ed
from dbnn.dbnn import DBNN
from dbnn.och import OCH

In [2]:
savefig = False

plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

if not savefig:
    plt.rcParams["figure.figsize"] = (4, 4)
    plt.rcParams["font.size"] = 15
    plt.rcParams["figure.titlesize"] = 25
    plt.rcParams["axes.labelsize"] = 20
    plt.rcParams["xtick.labelsize"] = 15
    plt.rcParams["ytick.labelsize"] = 15
    plt.rcParams["legend.fontsize"] = 13
    plt.rcParams["lines.linewidth"] = 2
else:
    plt.rcParams["figure.figsize"] = (8, 8)
    plt.rcParams["font.size"] = 30
    plt.rcParams["axes.labelsize"] = 53
    plt.rcParams["xtick.labelsize"] = 40
    plt.rcParams["ytick.labelsize"] = 40
    plt.rcParams["legend.fontsize"] = 28
    plt.rcParams["lines.linewidth"] = 4

Constants and hyperparameters:

In [3]:
# paths
dataset_root = 'datasets/'
dnn_path = 'models_checkpoints/dnn'
bnn_path = 'models_checkpoints/bnn'

# experiments
seed = 30
tf.random.set_seed(seed)
np.random.seed(seed)

# och parameters
och_x1_params = {'k': 5, 'l': 5.0, 's': 1.0}
och_x_params = {'k': 5, 'l': 1.0, 's': 1.0}
och_y_params = {'k': 5, 'l': 1.0, 's': 1.0}

# style
alpha = 0.12
colors = ["tab:blue", "tab:green", "tab:purple", "tab:red"]
labels = ["DNN", "MU", "DU", "DBNN"]
guide_linestyle=(0, (1, 1))
linestyles = [(0, (5, 1)), 'solid', (0, (5, 1)), 'solid']

In [4]:
def train_epoch(model, x_train, y_train, batch, optimizer, loss_ftn, *loss_metrics):
    indexes_batch = np.array_split(np.random.permutation(len(x_train)), len(x_train) / batch + 1)
    indexes_batch = [indexes for indexes in indexes_batch if len(indexes) > 0]

    for indexes in indexes_batch:
        xs = tf.stack([x_train[index] for index in indexes])
        ys = tf.stack([y_train[index] for index in indexes])
        train_step(model, xs, ys, optimizer, loss_ftn, *loss_metrics)
    
def train_step(model, x_batch, y_batch, optimizer, loss_ftn, *loss_metrics):
    with tf.GradientTape() as tape:
        losses = loss_ftn(model, x_batch, y_batch)
    gradients = tape.gradient(losses[0], model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    for loss, loss_metric in zip(losses, loss_metrics):
        loss_metric(loss)
    
def dnn_loss_ftn(model, xs, ys, loss_obj):
    preds = model(xs)    
    return loss_obj(ys, preds),

def bnn_normal_loss_ftn(model, xs, ys, length):
    logits = [tf.expand_dims(logit, axis=-1) for logit in tf.unstack(model(xs), axis=-1)]
    nll = - tf.reduce_mean(tfd.Normal(loc=logits[0], scale=tf.math.softplus(logits[1])).log_prob(ys))
    kl = sum(model.losses) / length
    loss = nll + kl
    return loss, nll

def bnn_categorical_loss_ftn(model, xs, ys, length):
    logits = model(xs)
    nll = - tf.reduce_mean(tfd.Categorical(logits=logits).log_prob(ys))
    kl = sum(model.losses) / length
    loss = nll + kl
    return loss, nll

In [5]:
# def dnn_op(model, x):
#     preds = model(tf.stack([x]))
#     return preds

def bnn_normal_op(model, x):
    logits = tf.unstack(model(tf.stack([x])), axis=-1)
    return tfd.Normal(loc=logits[0], scale=tf.math.softplus(logits[1])).sample()

def bnn_categorical_op(model, x):
    logits = model(x)
    return tfd.Categorical(logits=logits).log_prob(ys).sample()

Table of Contents
* [Toy Example of Regression](#Toy-Example-of-Regression)
* [Occupancy Detection Data Set (Classification)](#Occupancy-Detection-Data-Set-(Classification))

## Toy Example of Regression

In [ ]:
TOY_REGRESSION_FLAG = "toy-regression"

In [ ]:
def create_dnn():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation=tf.nn.relu),
        tf.keras.layers.Dense(1)
    ])

def create_bnn():
    return tf.keras.Sequential([
        tfp.layers.DenseFlipout(10, activation=tf.nn.relu),
        tfp.layers.DenseFlipout(2)
    ])

Generate datasets:

In [ ]:
num = 50
x_train = np.linspace(-3, 3, num=num)
y_train = np.cos(x_train) + np.random.normal(0, 0.1, size=num)
x_train = x_train.astype(np.float32).reshape((num, 1))
y_train = y_train.astype(np.float32).reshape((num, 1))

fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_xlim(-5.2, 5.2)
ax.set_ylim(-3.2, 3.2)
ax.scatter(x_train, y_train, color="black", s=30, marker="+", label="Training")

### Train Models

Train DNN:

In [ ]:
epochs, batch = 150, 10

dnn = create_dnn()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_obj = tf.keras.losses.MSE
loss_ftn = lambda model, xs, ys: dnn_loss_ftn(model, xs, ys, loss_obj)
train_loss = tf.keras.metrics.Mean(name='train_loss')

for epoch in range(epochs):
    train_epoch(dnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss)
    
    if (epoch + 1) % 30 == 0:
        template = 'Epoch {}, Loss: {}'
        print(template.format(epoch + 1,
                              train_loss.result()))
        train_loss.reset_states()
        
dnn.save_weights("%s-%s" % (dnn_path, TOY_REGRESSION_FLAG), save_format='tf')

Train BNN:

In [ ]:
epochs, batch = 300, 10

bnn = create_bnn()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_ftn = lambda model, xs, ys: bnn_normal_loss_ftn(model, xs, ys, len(x_train))
train_loss = tf.keras.metrics.Mean(name='train_loss')
nll_loss = tf.keras.metrics.Mean(name='nll_loss')

for epoch in range(epochs):
    train_epoch(bnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss, nll_loss)
        
    if (epoch + 1) % 30 == 0:
        template = 'Epoch {}, Loss: {}, NLL: {}'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              nll_loss.result()))
        train_loss.reset_states()
        nll_loss.reset_states()
        
bnn.save_weights("%s-%s" % (bnn_path, TOY_REGRESSION_FLAG), save_format='tf')

### Test Models

In [ ]:
x_pred = np.linspace(-5, 5, num=50)
x_pred = x_pred.astype(np.float32).reshape((50, 1))

DNN:

In [ ]:
dnn = create_dnn()
dnn.load_weights("%s-%s" % (dnn_path, TOY_REGRESSION_FLAG))

dnn_x_pred, dnn_y_pred, dnn_runtimes = [], [], []
for x in x_pred:
    time1 = time.time()
    dnn_x_pred.append(x)
    dnn_y_pred.append(dnn(tf.stack([x]))[0].numpy())
    time2 = time.time()
    dnn_runtimes.append(time2 - time1)
    
print("DNN Runtime: %.3f ± %.3f (ms)" % (np.mean(dnn_runtimes) * 10 ** 3, np.std(dnn_runtimes) * 10 ** 3))

MU:

In [ ]:
sample_no = 5
bnn = create_bnn()
bnn.load_weights("%s-%s" % (bnn_path, TOY_REGRESSION_FLAG))

mu_x_pred, mu_y_pred, mu_runtimes = [], [], []
for x in x_pred:
    time1 = time.time()
    for _ in range(sample_no):
        mu_pred = tf.unstack(bnn(tf.stack([x])), axis=-1)
        mu_x_pred.append(x)
        mu_y_pred.append(tfd.Normal(loc=mu_pred[0], scale=tf.math.softplus(mu_pred[1])).sample()[0].numpy())
    time2 = time.time()
    mu_runtimes.append(time2 - time1)
    
print("MU Runtime: %.3f ± %.3f (ms)" % (np.mean(mu_runtimes) * 10 ** 3, np.std(mu_runtimes) * 10 ** 3))

DU:

In [ ]:
sample_no = 5

x_dims, y_dims = [1], [1]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
du = DBNN(lambda x: dnn(tf.stack([x[0]])), och_x_1, och_x, och_y)

du_x_pred, du_y_pred, du_runtimes = [], [], []
for x in x_pred:
    time1 = time.time()
    du.update(x)
    time2 = time.time()
    du_runtimes.append(time2 - time1)
    for _ in range(sample_no):
        _x_pred, _y_pred = du.och_x.sample(), du.och_y.sample()
        if _x_pred is not None and _y_pred is not None:
            du_x_pred.append(_x_pred[0])
            du_y_pred.append(_y_pred.numpy())

print("DU Runtime: %.3f ± %.3f (ms)" % (np.mean(du_runtimes) * 10 ** 3, np.std(du_runtimes) * 10 ** 3))

DBNN:

In [ ]:
sample_no = 5

x_dims, y_dims = [1], [1]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
dbnn = DBNN(lambda x: bnn_normal_op(bnn, x[0]), och_x_1, och_x, och_y)

dbnn_x_pred, dbnn_y_pred, dbnn_runtimes = [], [], []
for x in x_pred:
    time1 = time.time()
    dbnn.update(x)
    time2 = time.time()
    dbnn_runtimes.append(time2 - time1)
    for _ in range(sample_no):
        _x_pred, _y_pred = dbnn.och_x.sample(), dbnn.och_y.sample()
        if _x_pred is not None and _y_pred is not None:
            dbnn_x_pred.append(_x_pred[0])
            dbnn_y_pred.append(_y_pred.numpy())

print("DBNN Runtime: %.3f ± %.3f (ms)" % (np.mean(dbnn_runtimes) * 10 ** 3, np.std(dbnn_runtimes) * 10 ** 3))

Visualize results:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(18, 4))
datasets = [(dnn_x_pred, dnn_y_pred), (mu_x_pred, mu_y_pred), (du_x_pred, du_y_pred), (dbnn_x_pred, dbnn_y_pred)]
for ax, preds, label, color in zip(axes, datasets, ["DNN", "MU", "DU", "DBNN"], colors):
    ax.set_xlim(-5.2, 5.2)
    ax.set_ylim(-3.2, 3.2)
    ax.scatter(x_train, y_train, color="black", s=30, marker="+", label="Training")
    ax.scatter(preds[0], preds[1], facecolors="none", edgecolors=color, label=label)
    ax.legend(edgecolor='black', fancybox=False)

## [Occupancy Detection Data Set](https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+) (Classification)

In [21]:
OCC_FLAG = "occupancy"
OCC_INPUT_DIM = 5
OCC_CLASS_NO = 2

In [22]:
train_file_names, test_file_names = ["datatraining.txt"], ["datatest.txt", "datatest2.txt"]

def occupancy_data(file_names):
    occupancy_path = dataset_root + "occupancy/"
    x, y = [], []
    for file_name in file_names:
        with open(occupancy_path + file_name, 'r') as csvfile:
            file_reader = csv.reader(csvfile, delimiter=',')
            header = next(file_reader)
            for row in file_reader:
                x.append(tf.constant([float(v) for v in row[2: 7]]))
                y.append(tf.constant(int(row[7])))
    return x, y

In [23]:
def create_dnn():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(50, activation=tf.nn.relu),
        tf.keras.layers.Dense(50, activation=tf.nn.relu),
        tf.keras.layers.Dense(OCC_CLASS_NO, activation=tf.nn.sigmoid)
    ])

def create_bnn():
    return tf.keras.Sequential([
        tfp.layers.DenseFlipout(50, activation=tf.nn.relu),
        tfp.layers.DenseFlipout(50, activation=tf.nn.relu),
        tfp.layers.DenseFlipout(OCC_CLASS_NO)
    ])

### Train Models

In [ ]:
x_train, y_train = occupancy_data(train_file_names)

Train DNN:

In [ ]:
epochs, batch = 10, 10

dnn = create_dnn()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy()
loss_ftn = lambda model, xs, ys: dnn_loss_ftn(model, xs, ys, loss_obj)
train_loss = tf.keras.metrics.Mean(name='train_loss')

for epoch in range(epochs):
    train_epoch(dnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss)
    
    if (epoch + 1) % 1 == 0:
        template = 'Epoch {}, Loss: {}'
        print(template.format(epoch + 1,
                              train_loss.result()))
        train_loss.reset_states()
        
dnn.save_weights("%s-%s" % (dnn_path, OCC_FLAG), save_format='tf')

Train BNN:

In [ ]:
epochs, batch = 20, 10

bnn = create_bnn()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_ftn = lambda model, xs, ys: bnn_categorical_loss_ftn(model, xs, ys, len(x_train))
train_loss = tf.keras.metrics.Mean(name='train_loss')
nll_loss = tf.keras.metrics.Mean(name='nll_loss')

for epoch in range(epochs):
    train_epoch(bnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss, nll_loss)
        
    if (epoch + 1) % 1 == 0:
        template = 'Epoch {}, Loss: {}, NLL: {}'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              nll_loss.result()))
        train_loss.reset_states()
        nll_loss.reset_states()
        
bnn.save_weights("%s-%s" % (bnn_path, OCC_FLAG), save_format='tf')

### Test Models

In [16]:
x_test, y_test = occupancy_data(test_file_names)

print("Testset size: ", len(x_test))

Testset size:  12417


In [14]:
def test_step(model, xs, ys, sampling, runtime_metric, *loss_metrics):
    for loss_ftn, metric in loss_metrics:
        time1 = time.time()
        sws = sampling(model, xs)
        time2 = time.time()
        runtime_metric(time2 - time1)
        for loss_ftn, metric in loss_metrics:
            losses = [tf.keras.losses.MSE(tf.one_hot(ys, OCC_CLASS_NO), s) * w for s, w in sws]
            sum(losses)

            loss = loss_ftn(sws, ys)
            metric(loss)

def test(model, x_test, y_test, batch, sampling, runtime_metric, *loss_metrics):
    indexes_batch = np.array_split(range(len(x_test)), len(x_test) / batch + 1)
    indexes_batch = [indexes for indexes in indexes_batch if len(indexes) > 0]
    for indexes in indexes_batch:
        xs = tf.stack([x_test[index] for index in indexes])
        ys = tf.stack([y_test[index] for index in indexes])
        test_step(model, xs, ys, sampling, runtime_metric, *loss_metrics)
        
mse_ftn = lambda sws, ys: sum([tf.keras.losses.MSE(tf.one_hot(ys, OCC_CLASS_NO), s) * w for s, w in sws])
cce_object = tf.keras.losses.SparseCategoricalCrossentropy()
cce_ftn = lambda sws, ys: sum([cce_object(ys, s) * w for s, w, in sws])

Load models:

In [11]:
dnn = create_dnn()
dnn.load_weights("%s-%s" % (dnn_path, OCC_FLAG))

bnn = create_bnn()
bnn.load_weights("%s-%s" % (bnn_path, OCC_FLAG))

Test DNN:

In [33]:
batch = 1

sampling = lambda model, xs: [(model(xs), 1.0)]

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')
cce_metric = tf.keras.metrics.Mean(name='cce')

test(dnn, x_test[:1000], y_test[:1000], batch, sampling, runtime_metric, (mse_ftn, mse_metric), (cce_ftn, cce_metric))
        
template = 'DNN Runtime: {} (ms), MSE: {}, NLL: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result(),
                      cce_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()
cce_metric.reset_states()

DNN Runtime: 0.8591588735580444 (ms), MSE: 0.11299999803304672, NLL: 0.15665099024772644


Test MU:

In [28]:
batch, sample_no = 1, 5

sampling = lambda model, xs: [(tf.one_hot(tfd.Categorical(logits=model(xs)).sample(), OCC_CLASS_NO), 1 / sample_no) for _ in range(sample_no)]

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')
cce_metric = tf.keras.metrics.Mean(name='cce')

test(bnn, x_test[:100], y_test[:100], batch, sampling, runtime_metric, (mse_ftn, mse_metric), (cce_ftn, cce_metric))

template = 'MU Runtime: {} (ms), MSE: {}, NLL: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result(),
                      cce_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()
cce_metric.reset_states()

Runtime: 129.5395050048828, MSE: 0.25700029730796814, NLL: 4.1423563957214355


Test DU:

In [27]:
batch = 1

x_dims, y_dims = [OCC_INPUT_DIM], [OCC_CLASS_NO]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
du = DBNN(lambda x: dnn(tf.stack([x[0]])), och_x_1, och_x, och_y)

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')
cce_metric = tf.keras.metrics.Mean(name='cce')

def sampling(model, xs):
    model.update(xs)
    return model.och_y.cws()

test(du, x_test[:100], y_test[:100], batch, sampling, runtime_metric, (mse_ftn, mse_metric), (cce_ftn, cce_metric))

template = 'DU Runtime: {} (ms), MSE: {}, NLL: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result(),
                      cce_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()
cce_metric.reset_states()

Runtime: 8.906378746032715, MSE: 0.45500001311302185, NLL: 0.630763053894043


Test DBNN:

In [31]:
batch = 1

x_dims, y_dims = [OCC_CLASS_NO], [OCC_CLASS_NO]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
dbnn = DBNN(lambda x: bnn_categorical_op(bnn, x[0]), och_x_1, och_x, och_y)

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')
cce_metric = tf.keras.metrics.Mean(name='cce')

def sampling(model, xs):
    model.update(xs)
    return model.och_y.cws()

test(du, x_test[:100], y_test[:100], batch, sampling, runtime_metric, (mse_ftn, mse_metric), (cce_ftn, cce_metric))

template = 'DBNN Runtime: {} (ms), MSE: {}, NLL: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result(),
                      cce_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()
cce_metric.reset_states()

DBNN Runtime: 11.921072006225586 (ms), MSE: 0.48750370740890503, NLL: 1.1915442943572998


## [EMG Data for Gestures Data Set](https://archive.ics.uci.edu/ml/datasets/EMG+data+for+gestures) (Classification)

In [ ]:
classes_no = 8

def emg_data(dataset_no):
    emg_path = dataset_root + "emg/"
    channels, labels = [], []
    dat_path = emg_path + "%02d/" % dataset_no
    for file_name in os.listdir(dat_path):
        with open(dat_path + file_name, 'r') as csvfile:
            file_reader = csv.reader(csvfile, delimiter='\t')
            header = next(file_reader)
            for row in file_reader:
                if len(row) == 10:
                    channels.append(tf.constant([float(v) for v in row[1:8]]))
                    labels.append(tf.constant([float(row[9])]))
    return channels, labels

### Train Models

Train DNN:

Train BNN:

### Test Models

Test DNN:

Test MU:

Test DU:

Test DBNN: